📈 Stock Price Prediction
===================================

✍️ Author: Shah Ahmad Noorani


🔗 Connect with Me:

💼 LinkedIn:
https://www.linkedin.com/in/shah-ahmad-noorani-5804b0278/

💻 GitHub:
https://github.com/shahahmad-dev

📊 Kaggle:
https://www.kaggle.com/shahahmadnoorani

**`Stock Market Data scrapping in Python`**

In [ ]:
# imporot libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import yfinance as yf
import datetime as dt
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# define the date to fetch the data
# define the dates to fetch data
today = date.today()
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2
print(f'The data is started from {start_date} to {end_date}')

In [ ]:
# deifne the ticker
ticker = 'GOOGL'

df = yf.download(ticker, start=start_date, end=end_date, progress=False)

In [ ]:
# make date numaric
df.insert(0, "Date", df.index, True)
# df["Date"] = df.index
df.reset_index(drop=True, inplace=True)

df.columns = df.columns.get_level_values(0)

In [ ]:
#  reset columns sequence
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

In [ ]:
# remove the name column
df.columns.name = None

In [ ]:
df.describe()

In [ ]:
fig = px.line(df, x='Date', y=df.columns, title='Stock Price')
fig.show()

In [ ]:
# select columns
df1 = df[['Date', 'Close']]
df1.head()

In [ ]:
# plot the to see how it look like
fig = px.line(df1, x='Date', y='Close', title='Google Stock Price')
fig.show()

In [ ]:
# Check the data is Statinority or not
from statsmodels.tsa.stattools import adfuller
def check_stationarity(df):
  result = adfuller(df)
  print('ADF Statistic: %f'% result[0])
  print('P-value: %f'% result[0])
  if result[1]<= 0.05:
    print('Data is Stationary')
  else:
    print("Data isn't Stationary")

In [ ]:
check_stationarity(df['Close'])

In [ ]:
# Decompose the data to see the trend
from statsmodels.tsa.seasonal import seasonal_decompose
decompose = seasonal_decompose(df['Close'], model='additive', period=30)
decompose.plot()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import matplotlib.pyplot as plt

# Original series
fig, axes = plt.subplots(3,2, sharex=True)
axes[0,0].plot(df['Close']); axes[0,0].set_title('Original series')
plot_acf(df['Close'], ax = axes[0,1])

# 1st Difference
axes[1,0].plot(df['Close'].diff().dropna())
plot_acf(df['Close'].diff().dropna(), ax=axes[1,1])

# 2nd Difference
axes[2,0].plot(df['Close'].diff().diff().dropna())
plot_acf(df['Close'].diff().diff().dropna(), ax=axes[2,1])

plt.show()


# **`d = 1`**

# **Let's find the `p ` value**

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
# pd.plotting.autocorrelation_plot(df['Close'])

plot_acf(df['Close'], alpha=0.05)

from statsmodels.tsa.stattools import acf, pacf
x_acf = pd.DataFrame(acf(df['Close']))
print(x_acf)
fig.show()

# **`p = 7`**

# **Let's find the `q ` value**

In [ ]:
# Partial autocorrelation
from statsmodels.tsa.stattools import acf, pacf
plot_pacf(df['Close'], lags=20, alpha=0.05)
fig.show()


# **`q = 2`**

In [ ]:
# let's define p, d, q
p = 7
q = 2
d = 1

In [ ]:
# !pip install pmdarima

# let's run arima with pmdarima
from pmdarima.arima import auto_arima

model = auto_arima(df['Close'], start_p=7, start_q=2, max_p=14,
                   max_q=5, m=12, start_P=0,
                   seasonal=True, d=1, D=1,
                   trace=True, error_action='ignore',
                   suppress_warning=True)

In [ ]:
model = auto_arima(df['Close'], seasonal=True, suppress_warnings=True)
print(model.summary())

In [ ]:
 # stats model and arima
 from statsmodels.tsa.arima.model import ARIMA
p, d, q, = 0, 1, 0

model = ARIMA(df['Close'], order=(p, d, q))
model = model.fit()
print(model.summary())

# Predict next 30 days
forecost = model.predict(n_periods=30)
print(forcost)

# plot the forcost
plt.figure(figsize=(10, 5))
plt.plot(df['Close'], label='Actual')
plt.plot(forecost, label='Forecast')
plt.show()

In [ ]:
# Let's run SERIMAX
import statsmodels.api as sm

p, d, q = 0, 1, 0

model = sm.tsa.statespace.SARIMAX(
    df['Close'],
    order=(p, d, q),
    seasonal_order=(p,d, q, 12)
)

model = model.fit()
print(model.summary())

In [ ]:
predictions = model.predict(start=len(df['Close']), end=len(df['Close'])+ 10)

# plot the forecost
plt.figure(figsize=(10, 5))
plt.plot(df['Close'], label='Actual')
plt.plot(predictions, label='Predicated')
plt.show()